# Earthquakes in Japan 2001-2018

**จัดทำโดย   นายเจตพล  เซาะจอหอ    1600900730
            นายชยุต   สมวงษ์อินทร์   1600900987



## Workflow stages


1. Question or problem definition.
2. Acquire training and testing data.
3. Wrangle, prepare, cleanse the data.
4. Analyze, identify patterns, and explore the data.
5. Model, predict and solve the problem.
6. Visualize, report, and present the problem solving steps and final solution.



## Question and problem definition

เนื่องจากประเทศญี่ปุ่นเกิดแผ่นดินไหวบ่อย แต่หากเราจำเป็นต้องอาศัยอยู่ในญี่ปุ่นอาจจะต้องมีข้อมูลของแต่ละเมืองหรือจุดที่จะอยู่ว่ามีแผ่นดินไหวมากน้อยรุนแรงแค่ไหน
จึงจะสร้าง Model ที่สามารถบอกได้ว่าพิกัดนี้มีความเสี่ยงในการเกิดแผ่นดินไหวมากน้อยแค่ไหน

In [ ]:
### เรียกใช้ Library และ Load data

import numpy as np
import pandas as pd 
import datetime

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import pandas as pd

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

train_data = pd.read_csv("../input/earthquakes-in-japan/Japan earthquakes 2001 - 2018.csv")
#train_data.rename(columns={'Unnamed: 0': 'datetime'}, inplace=True)

# Analyze by describing data

In [ ]:
print(train_data.columns.values) # ดู Columnว่ามีอะไรบ้าง

ดูข้อมูล่วนหัวตาราง

In [ ]:
train_data.head() #ดูข้อมูลในตาราง

ดูข้อมูลส่วนท้ายตาราง

In [ ]:
train_data.tail()

### Cleans the data.

#### เลือก column ข้อมูลที่มีประโยชน์

In [ ]:
train_data = train_data[['time','latitude', 'longitude', 'depth', 'mag','place']]

คัดเลือก Column 6 ตัวนี้เพราะตัวแปรอื่นๆใน Dataset เป็นค่าที่ไม่ได้ใช้งานและเป็นแค่ส่วนกำกับข้อมูลเท่านั้นเช่นค่า Spec ของเครื่องที่ใช้วัดสถานที่วัดและความคลาดเคลื่อน

### ดูค่าเฉลี่ย,ส่วนเบี่ยงแบน ในข้อมูล

In [ ]:
train_data.describe()

* ความรุนแรงของแผ่นดินไหว อยู่ที่ 4.8 ริกเตอร์
* ค่าเฉลี่ยความลึกของศูนย์กลางแผ่นดินไหว 51.36 km.
* ค่าส่วนเบี่ยงเบนมาตราฐานความลึกของศูนย์กลางแผ่นดินไหว = 76.603 km.
* พิกัดศูนย์กลางแผ่นดินไหวจากค่าเฉลี่ย latitude 37.410294 , longitude 142.980441

### แปลงค่าวันเวลาให้เป็น UNIX
##### เพื่อให้ง่ายต่อการวิเคราะห์ในขั้นตอนการสร้าง Model (Just in case)

In [ ]:
#จัด format 2001-01-04T04:18:21.430Z ให้เป็น 2001-01-04 04:18:21.430000+00:00
train_data['time'] = pd.to_datetime(train_data['time'],format= '%Y-%m-%dT%H:%M:%S') 
#แปลงเวลาเป็น UNIX Time Stamp เพื่อให้ง่ายต่อการจัดการ
train_data['timestamp'] = train_data.time.values.astype(np.int64) 

train_data.head()

In [ ]:
#หาวัน(ตัวเลข)และเพิ่มใน col
train_data['day_of_week'] = pd.to_datetime(train_data['time']).dt.weekday+1
#หาเดือนและเพิ่มใน col
train_data['month'] = pd.to_datetime(train_data['time']).dt.month
#หาปีและเพิ่มใน col
train_data['year'] = pd.to_datetime(train_data['time']).dt.year



### จากข้อมูลที่มีแยกประเภท Data ได้ดังนี้ ### 

**ประเภทของ Column ที่เป็นตัวเลข Categorical**
* Categorical: day_of_week,day_of_week_name,month,year,place
****[](http://)
**ประเภทของ Column ที่เป็นตัวเลข**
* Continous:  time,latitude,longitude,depth,mag.

### หาข้อมูลว่าในแต่ละเดือนในแต่ละปีมีแผ่นดินไหวบ่อยขนาดไหน ### 
จากข้อมูลที่มีจะเลือกการแสดงผลแบบ Heatmap โดยให้แกนนอนเป็นปี และ แกนตั้งเป็นเดือน

In [ ]:
## ทำ Pivot จากปีและเดือนโดยนับค่าครั้งที่เกิด
pt = train_data.pivot_table(index="month",columns="year",values="mag", aggfunc="count") 
#กำหนดขนาด และสี Heatmap 
fig, ax = plt.subplots(figsize=(8,6)) 
#แสดง Heatmap
sns.heatmap(pt,cmap="Blues",annot=True,fmt='g') 


จาก Heatmap แสดงให้เห็นว่าในปี 2011 เดือน 3 นั้นมีจำนวนครั้งของแผ่นดินไหนที่เยอะมากถึง 1984 ครั้งและในเดือนอื่นๆของปีนั้นก็มีค่ามาก ทำให้ค่ารวมในปีนั้นสูง
จึงจะทำการแยกค่าของปี 2011 ออก

In [ ]:
### เอาข้อมูลปี 2011 ออก
train_data = train_data.drop(train_data[train_data.time.dt.year == 2011].index)
## ทำ Pivot และ Plot อีกครั้ง
## ทำ Pivot จากปีและเดือนโดยนับค่าครั้งที่เกิด
pt = train_data.pivot_table(index="month",columns="year",values="mag", aggfunc="count") 
#กำหนดขนาด และสี Heatmap 
fig, ax = plt.subplots(figsize=(8,6)) 
#แสดง Heatmap
sns.heatmap(pt,cmap="Blues",annot=True,fmt='g') 





จากการตัดปี 2011 ออก Data โดยรวมดูใกล้เคียงกัน

### หาว่าแต่ละวันวันไหนมีโอกาสเกิดแผ่นดินไหวเยอะสุด ###

In [ ]:
#ทำการนับแถวที่เกิดขึ้นในแต่ละวันแล้วนำค่าที่นับได้ในแต่ละวันมาคิดเป็น % ของทั้ง 7 วัน
train_data[['day_of_week','mag']].groupby(['day_of_week']).count().rename(columns={"mag":"% of count"}).apply(lambda x: 100 * x / float(x.sum())).round(2).rename(index={
    1:"Monday",2:"Tuesday",3:"Wednesday",4:"Thursday",5:"Friday",6:"Saturday",7:"Sunday"
})

จากข้อมูลตั้งแต่ปี 2001-2018 ยกเว้นปี 2011 จะเห็นได้ว่าแต่ละ วันในสัปดาห์มีโอกาสเกิดแผ่นดินไหวใกล้เคียงกัน

### หาว่าแต่ละเดือน เดือนไหนมีโอกาสเกิดแผ่นดินไหวเยอะสุด

In [ ]:
#ทำการนับแถวที่เกิดขึ้นในแต่ละเดือนแล้วนำค่าที่นับได้ในแต่ละเดือนมาคิดเป็น % ของทั้ง 7 วัน
train_data[['month','mag']].groupby(['month']).count().rename(columns={"mag":"% of count"}).apply(lambda x: 100 * x / float(x.sum())).round(2).rename(index={
    1:"January",2:"February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"
})

จากตารางวิเคราะห์หาการเกิดแผ่นดินไหวตามเดือนที่เกิดขึ้น พบว่ามีการเกิดแผ่นดินไหวใกล้เคียงกันและช่วงท้ายปีเดือน 11-12 มีค่าเยอะกว่าช่วงกลางปีเท่าตัว แต่เมื่อย้อนไปดูข้อมูลภาพรวมแต่ละปีด้านบนข้อมูลเพิ่มมากจากเดือน 11/2006 , 12/2010 ,12/2012

### **หาว่าแผ่นดินไหวกลางวันหรือกลางคืนมากกว่ากัน** ###



In [ ]:
## ทำ feature แบ่งช่วงเวลาชั่วโมงเป็น 6 ช่วงเวลา ให้ช่วงเวลา 0-4 เป็น LateNight 4-8 เป็น Early Morning เป็นต้นแล้วเพิ่มใน Column session
b = [0,4,8,12,16,20,24]
l = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']
train_data['session'] = pd.cut(pd.to_datetime(train_data['time']).dt.hour, bins=b, labels=l).replace(np.nan, 'Night') ##np.nan คือเวลาชั่ว 0 เป็น NaN จึงแทนด้วย Night 
## ทำ Pivot เพื่อดูว่าเวลาแต่ละช่วงเกิดแผ่นดินไหวเท่าไรโดยนับจำนวนครั้งที่เกิดในแต่ละเวลาแล้วนำมาคิดว่าช่วงเวลานั้นเป็นกี่ % เมื่อเทียบกับช่วงเวลาอื่นทั้งหมด
train_data[['session','mag']].groupby(['session']).count().rename(columns={"mag":"% of count"}).apply(lambda x: 100 * x / float(x.sum())).round(2)

จากการรันแต่ละช่วงเวลาที่เกิดแผ่นดินไหวนั้นมีอัตราส่วนที่เท่าๆ

### สรุปในส่วนนี้จากข้อมูลที่ Explore การเกิดแผ่นดินไหวนั้นเป็นสิ่งที่ไม่แน่นอนมีโอกาสเกิดขึ้นได้ตลอดไม่มี Pattern 

# Analyze by Visualizing data

### Heatmap ของจุดที่เกิดแผ่นดินไหว

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap

map_hooray = folium.Map(location=[38.495035, 137.026582],
                    zoom_start = 4) 

# Ensure you're handing it floats
#xx = train_data.loc[train_data['year'] == 2012]
xx = train_data
xx['latitude'] = xx['latitude'].astype(float)
xx['longitude'] = xx['longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs

xx = xx[['latitude', 'longitude']]
xx = xx.dropna(axis=0, subset=['latitude','longitude'])

# List comprehension to make out list of lists
xx = [[row['latitude'],row['longitude']] for index, row in xx.iterrows()]

# Plot it on the map
HeatMap(xx,min_opacity=0.4,radius=16, blur=10).add_to(map_hooray)

# Display the map
map_hooray

จากแผนภาพตำแหน่งที่เกิดแผ่นดินไหวนำมา plot บน HeatMap เมื่อซูมเข้าไปจุดสีแดงคือบริเวณนั้นเกิดแผ่นดินไหวหลายครั้ง โดยส่วนมากเกิดขึ้นในทะเลมากกว่าบนบก ตำแหน่งที่เกิดบนบกเยอะคือเมือง tokyo และเมืองทางตอนใต้ของแผ่นดิน รูปทรงตำแหน่งมีแนวของจุกที่เกิดแผ่นดินไหวอยู่ตามแนวเกาะเล็กๆในทะเล และตำแหน่งแผ่นดินไหวจะเกิดในตะวันออกของญี่ปุ่นมากกว่า

### Chart ของความแรงแผ่นดินไหวว่าแต่ละระดับเกิดขึ้นมากน้อยเพียงใด

In [ ]:
mgchart = train_data[['month','mag']].groupby(['mag']).count().rename(columns={"month":"count"})
sns.distplot(train_data['mag'], hist=True, kde=True, 
             bins=40, color = 'blue',
             hist_kws={'edgecolor':'black'})
mgchart.head()


ความแรงที่เกิดขึ้นส่วนมากอยู่ที่ระดับ 4.5-5 ริกเตอร์ เราอาจจะต้องแบ่งประเภทของความรุนแรง(mag)เป็นช่วงแล้วนำไปทำ categorize เพื่อไปใช้ใน Model
* 4.0 - 4.9 แผ่นดินไหวขนาดค่อนข้างเล็ก (Light)    = 1
* 5.0 - 5.9 แผ่นดินไหวขนาดปานกลาง (Moderate)  = 2
* 6.0 - 6.9 แผ่นดินไหวขนาดค่อนข้างใหญ่ (Strong)  = 3
* 7.0 - 7.9 แผ่นดินไหวขนาดใหญ่ (Major)         = 4
* มากกว่า 8.0 แผ่นดินไหวใหญ่มาก (Great)          = 5

In [ ]:
# ทำ feature โดยใช้ pd.cut แบ่งช่วงตามด้านบน
bins = [4, 5, 6, 7, 8, np.inf]
names = ['1', '2', '3', '4', '5']
train_data['mag_cat'] = pd.cut(train_data['mag'], bins, labels=names)

### หาจำนวนครั้งที่เกิดในแต่ละระดับ 

In [ ]:
train_data[['mag_cat','place']].groupby(['mag_cat']).count().rename(columns={"mag":"% of count"}).rename(columns={"place":"count"})

จำนวนครั้งที่เกิดแผ่นดินไหวในความรุนแรงแต่ละระดับจะเห็นได้ว่ามีแผ่นดินไหวที่มีความรุนแรงมาเกิดขึ้นน้อยครั้ง

### นำความรุนแรงแต่ละระดับมาดูใน Heatmap ว่าจุดที่ความรุนแรงมากอยู่ตำแหน่งไหน

In [ ]:
map_hooray2 = folium.Map(location=[38.495035, 137.026582],
                    zoom_start = 4) 
## ทดลองเลือกความรุนแรงระดับ 4 และ 5
yy = train_data.loc[train_data['mag_cat'].isin(['4','5'])]
yy['latitude'] = yy['latitude'].astype(float)
yy['longitude'] = yy['longitude'].astype(float)

yy = yy[['latitude', 'longitude']]
yy = yy.dropna(axis=0, subset=['latitude','longitude'])

yy = [[row['latitude'],row['longitude']] for index, row in yy.iterrows()]

HeatMap(yy,min_opacity=0.8,radius=20, blur=10).add_to(map_hooray2)

map_hooray2

จาก heatmap คือจุดที่เกิดแผ่นดินไหว cat 4 และ 5 มีบางจุดสีแดงเป็นจุดที่เกิดขึ้นซ้ำดังนั้นพื้นที่ๆใกล้เคียงจุดที่รุนแรงจะเป็นอันตราย จากตรงนี้จึงนำ Lat/Long มาใช้เป็นเป็นตัวแปรในการทำ Model เพื่อหาว่าจุดใดเสี่ยงมากน้อยเพียงใด

# Model 
จากการหาข้อมูลต่างๆด้านบนจึงเลือกออกแบบ Model การคาดเดาความรุนแรงของแผ่นดินไหวที่อาจจะเกิดขึ้นในพิกัดนั้นๆ[Lat,Long] เพื่อให้ผู้ที่ต้องการอยู่อาศัยได้เช็คว่าตัวเองอยู่ในพื้นที่ความเสี่ยงระดับใด [mag_cat] รูปแบบการทำ ML แบบ supervised จากข้อมูลเลือกใช้ Model แบบ Classification เนื่องจากค่าที่ต้องการให้ Model Predict ออกมาเป็นระดับความรุนแรงที่แบ่งไว้ 5 ระดับ 1-5 

We are also perfoming a category of machine learning which is called supervised learning as we are trai

## ขั้นตอน 
1. แบ่ง Test และ Train Data
2. เริ่มจากการ Train Model หลายๆแบบ
3. หาค่า Performane 
4. เปรียบเทียบ Performace แต่ละแบบ
5. เลือก Model ว่าจะใช้ Model แบบไหนในการใช้งาน
6. นำ Model ไปใช้งาน

### แบ่ง Train_data และ Test_data

In [ ]:
X = train_data[['latitude', 'longitude']]
y = train_data[['mag_cat']]


from sklearn.model_selection import train_test_split  
#แบ่งข้อมูล test train ใช้อัตราส่วน train 80 test 20 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=44)
print(X.shape)
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)

**จากข้อมูล 10281 แถวทำการแบ่งข้อมูลโดยใช้ train_test_split อัตราส่วน TrainData 70% = 8224แถว (2057, 2) และ TestData 30% = 2057แถว

## เริ่ม Train Model หลายๆแบบ

### Support Vector Machines Model
เลือกใช้ RBF kernel (Radial Basis Function) เนื่องจากลักษณะการกระจายคล้ายกับรัศมีของแผ่นดินไหว
![](https://scikit-learn.org/stable/_images/sphx_glr_plot_svm_kernels_003.png)

In [ ]:
# Support Vector Machines
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train) * 100, 2)
acc_svc

ได้คะแนน 82.99% อยู่ในเกณฑ์ที่นำไปใช้ได้

### Linear SVC

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, y_train) * 100, 2)
acc_linear_svc

ได้คะแนน 38.07% อยู่ในเกณฑ์ต่ำ model loose

## RandomForest Model

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

ได้คะแนน 99.99% พบว่า Model นี้ fit จนเกินไป

### KNN (k nearest neighbors) classification

เลือก Model ประเภทนี้มาพิจารณาเพราะพิกัดแผนที่แต่ละจุดนั้นจะเป็น Contour คล้ายภาพ

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_classification_001.png)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred_select = knn.predict(X_test)
acc_knn = round(knn.score(X_train, y_train) * 100, 2)
acc_knn

ได้คะแนน 86.21% อยู่ในเกณฑ์ที่นำไปใช้ได้

### LGBM Classifier

In [ ]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(objective='multiclass', random_state=44)
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)
acc_lgbm = round(lgbm.score(X_train, y_train) * 100, 2)
acc_lgbm

ได้คะแนน 83.48% อยู่ในเกณฑ์ที่นำไปใช้ได้ และได้ทำการเลือกโมเดลนี้โดยเปลี่ยนค่าตัวแปรเป็น y_pred_select เพื่อนำไปแสดงในตาราง output

### สรุปเปรียบเทียบ Socre ของ Model แต่ละแบบ

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 
              'Random Forest', 'LightGBM', 'Linear SVC'],
    'Score': [acc_svc, acc_knn,
              acc_random_forest, acc_lgbm, acc_linear_svc]})
models.sort_values(by='Score', ascending=False)

จากคะแนนจากแต่ละ Model  KNN, LightGBM , SVM  อยู่ในเกณฑ์ ที่นำไปใช้ได้ Random Forreset Socre เยอะเกินไป จึงได้
* เลือก KNN เป็น Model ในการแสดงผล Model Output เพราะได้ Performance สูงสุดในกลุ่มและไม่ Fit เกินไป 
* SVM ผลลัพท์ที่ได้จาก Model เป็นค่าเดียวกันทั้งหมด (Mag_cat = 1) อาจจะเป็นเพราะจำนวนของ train_data ช่วง mag 4-5 มีเยอะดัง Histrogram
* คะแนนต่ำเพราะเป็นรูปแบบเชิงเส้นไม่เหมาะกับข้อมูลลักษณะนี้

### Model Output

นำ test_Data ที่ผ่านการรันใน Model และได้ output ใส่ dataframe final และทำการแปลงระดับความแรง Mag_rate ให้เป็นตัวหนังสือเพื่อให้ง่ายต่อการอ่าน

In [ ]:
final = pd.DataFrame({
        "Latitude": X_test["latitude"],
        "Longitude": X_test["longitude"],
        "Mag_cat": y_pred_select,
        "Marker_color": y_pred_select
    })
# Map ตัวเลขเป็น Text
final["Mag_rate"] = final["Mag_cat"].replace({"1":"Light", "2": "Moderate", "3": "Strong", "4": "Major", "5": "Great"})
# ทำสีเพื่อแสดงใน visualization
final["Marker_color"] = final["Mag_cat"].replace({"1":"yellow", "2": "orange", "3": "red", "4": "purple", "5": "black"})
#เรียงจากมากไปน้อย
final.sort_values('Mag_rate', ascending=0).head(5000)

จากการรัน Model จะได้ข้อมูลค่าพิกัดต่างๆจาก Test_data ที่รันผ่าน Model แล้วจะได้ข้อมูลออกมาที่ Mag_rate ว่าจุดนั้นมีระดับการเกิดแผ่นดินไหวแรงในระดับใด

### นำผลจากการรัน Model มาแสดงในแผนที่

In [ ]:
import geopandas as gpd

import folium
from folium import *
from folium.plugins import *

import os
from pathlib import Path

map_hooray3 = folium.Map(location=[38.495035, 137.026582],
                    zoom_start = 4) 
## เพิ่มวงกลมสีในแผนที่
for idx, e in final.iterrows():
    Circle(location=[e['Latitude'], e['Longitude']], radius=1500, color=e['Marker_color']).add_to(map_hooray3)

map_hooray3

จากแผนที่วงกลมสีเหลืองจะเป็นพื้นที่ๆอยู่ในระดับ 1 Light จะมีสีส้มและแดงบ้างที่เป็น Moderate และ Strong

### ทำการใส่ค่าแบบ Manual โดยใช้ค่าพิกัดที่ใกล้เคียงกับจุดในข้อมูล Test เพื่อดูผลลัพท์

In [ ]:
data =  [[35.689273, 139.701653],[34.392609, 132.462899],[34.654216, 135.550717],[37.467216, 138.867288],[42.687249, 141.697015],[24.798245, 125.295468]] 
manual_test = pd.DataFrame(data, columns = ['latitude', 'longitude']) 
out = knn.predict(manual_test)

manual_out = pd.DataFrame({
        "Latitude": manual_test["latitude"],
        "Longitude": manual_test["longitude"],
        "Mag_cat": out,
        "Mag_rate": out
    })
# Map ตัวเลขเป็น Text
manual_out["Mag_rate"] = manual_out["Mag_cat"].replace({"1":"Light", "2": "Moderate", "3": "Strong", "4": "Major", "5": "Great"})
manual_out["Marker_color"] = manual_out["Mag_cat"].replace({"1":"yellow", "2": "orange", "3": "red", "4": "purple", "5": "black"})
manual_out.sort_values('Mag_rate', ascending=0)

จากค่า Manual Long/Lat ผลการทดสอบ Model ทำงานได้โดยสามารถบอกถึงความเสี่ยงของแผ่นดินไหวในแต่ละจุดได้ตามที่เราต้องการ

### นำผลแบบ Manual plot ลงแผนที่

In [ ]:
map_hooray4 = folium.Map(location=[38.495035, 137.026582],
                    zoom_start = 5) 
## เพิ่มวงกลมสีในแผนที่
for idx, e in manual_out.iterrows():
    Circle(location=[e['Latitude'], e['Longitude']], radius=50000, color=e['Marker_color']).add_to(map_hooray4)

map_hooray4

สรุป Model นี้สามารถนำไปใช้งานในเบื้องต้นได้ หากต้องการให้แม่นยำกว่านี้อาจจะต้องนำตัวแปรเกี่ยวกับสถานที่ในแต่ละจังหวัดหรือเขตมาช่วยว่าตำแหน่งนี้อยู่ในจังหวัด/เขตอะไรแล้วเอาการนับการเกิดมาเทียบกับเขตนั้นและเพื่อให้เรื่องของจำนวนการเกิดนั้นมาเกี่ยวข้องในโมเดลนี้ด้วย โดย Dataset เพื่อให้เกิดความแม่นยำมากขึ้นและสามารถนำไปใช้งานได้จริง รวมไปถึงปัจจัยอื่นๆนอกเหนือจากนี้เช่นแผ่นดินไหวที่เกิดขึ้นในทะเลอาจจะมีความสี่ยงที่จะเกิดซึนามิได้